In [2]:
pip install keras-tqdm

In [3]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [4]:

import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
#from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import layers
from keras.layers import *
from keras.utils import np_utils

from keras_tqdm import TQDMNotebookCallback

In [5]:
#ignore warnings in the output
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1099927324619996221
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15703311680
locality {
  bus_id: 1
  links {
  }
}
incarnation: 702575247030263237
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



In [6]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
train = pd.read_csv('/content/gdrive/MyDrive/Internship/emnist/emnist-balanced-train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/Internship/emnist/emnist-balanced-test.csv')
mapping = pd.read_csv("/content/gdrive/MyDrive/Internship/emnist/emnist-balanced-mapping.txt", 
                      delimiter = ' ', 
                      index_col=0, 
                      header=None, 
                      squeeze=True)

In [8]:
train.shape, test.shape, mapping.shape

((112799, 785), (18799, 785), (47,))

In [9]:
X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]

del train
del test

In [10]:
classes = np.unique(y_train)
num_classes = len(classes)
num_classes

47

In [11]:
# Convert the images into 3 channels
X_train=np.dstack([X_train] * 3)
X_test=np.dstack([X_test]*3)
X_train.shape,X_test.shape

((112799, 784, 3), (18799, 784, 3))

In [12]:
# Reshape images as per the tensor format required by tensorflow
X_train = X_train.reshape(-1, 28,28,3)
X_test= X_test.reshape (-1,28,28,3)
X_train.shape,X_test.shape

((112799, 28, 28, 3), (18799, 28, 28, 3))

In [13]:
# Resize the images 48*48 as required by VGG16
from keras.preprocessing.image import img_to_array, array_to_img
X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_train])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_test])
#train_x = preprocess_input(x)
X_train.shape, X_test.shape

((112799, 48, 48, 3), (18799, 48, 48, 3))

In [15]:
# Normalise the data and change data type
X_train = X_train / 255.
X_test = X_test / 255.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [16]:
from keras.utils import to_categorical

# Converting Labels to one hot encoded format
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [17]:
# partition to train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train, 
                                                  test_size= 0.10, 
                                                  random_state=88)

In [18]:
# Finally check the data size whether it is as per tensorflow and VGG16 requirement
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((101519, 48, 48, 3), (11280, 48, 48, 3), (101519,), (11280,))

In [19]:
# Define the parameters for instanitaing VGG16 model. 
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time
import matplotlib.pyplot as plt
#from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
#from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16;
from keras.applications.vgg16 import preprocess_input
import os

In [21]:
# Preprocessing the input 
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [22]:
#  Create base model of VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False, 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH)
                 )
conv_base.summary()



58892288/58889256 [==============================] - 2s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
______________________________________________________________

In [23]:
# Extracting features
train_features = conv_base.predict(np.array(X_train), batch_size=BATCH_SIZE, verbose=1)
test_features = conv_base.predict(np.array(X_test), batch_size=BATCH_SIZE, verbose=1)
val_features = conv_base.predict(np.array(X_val), batch_size=BATCH_SIZE, verbose=1)
#for layer in conv_base.layers:
#    layer.trainable = False

705/705 [==============================] - 4s 6ms/step


In [24]:
# 6.1 Saving the features so that they can be used for future
np.savez("train_features", train_features, y_train)
np.savez("test_features", test_features, y_test)
np.savez("val_features", val_features, y_val)

In [25]:
# Current shape of features
print(train_features.shape, "\n",  test_features.shape, "\n", val_features.shape)

(101519, 1, 1, 512) 
 (18799, 1, 1, 512) 
 (11280, 1, 1, 512)


In [26]:
# Flatten extracted features
train_features_flat = np.reshape(train_features, (101519, 1*1*512))
test_features_flat = np.reshape(test_features, (18799, 1*1*512))
val_features_flat = np.reshape(val_features, (11280, 1*1*512))

In [27]:
from keras import models
from keras.models import Model
from keras import layers
from keras import optimizers
from keras import callbacks
from keras.layers.advanced_activations import LeakyReLU

# 7.0 Define the densely connected classifier followed by leakyrelu layer and finally dense layer for the number of classes
NB_TRAIN_SAMPLES = train_features_flat.shape[0]
NB_VALIDATION_SAMPLES = val_features_flat.shape[0]
NB_EPOCHS = 100

In [28]:
train_features_flat.shape, val_features_flat.shape, y_train.shape, y_val.shape

((101519, 512), (11280, 512), (101519,), (11280,))

In [29]:
class VGG16:
    @staticmethod
    def build(shape, classes):
        model = models.Sequential()
        model.add(layers.Dense(512, activation='relu', input_dim=(1*1*512)))
        model.add(layers.LeakyReLU(alpha=0.1))
        model.add(layers.Dense(47, activation='softmax'))
        return model

In [30]:
y_train = to_categorical(y_train, 47)
y_test = to_categorical(y_test, 47)
y_val = to_categorical(y_val, 47)

In [31]:
y_train.shape, y_test.shape, y_val.shape

((101519, 47), (18799, 47), (11280, 47))

In [32]:
def create_clients(image_list, label_list, num_clients=5, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [33]:
#create clients
clients = create_clients(train_features_flat, y_train, num_clients=10, initial='client')

In [34]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [46]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    print("processing")
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set
print("processing Test batches")  
test_batched = tf.data.Dataset.from_tensor_slices((val_features_flat, y_val)).batch(len(y_val))

processing
processing
processing
processing
processing
processing
processing
processing
processing
processing
processing Test batches


In [48]:
lr = 0.01 
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )              

In [37]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad

def test_model(test_features_flat, y_val,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(val_features_flat)
    loss = cce(y_val, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(y_val, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss


In [49]:
#initialize global model
smlp_global = VGG16()
global_model = smlp_global.build(512,47)
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = VGG16()
        local_model = smlp_local.build(512,47)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(test_features_flat, Y_val) in test_batched:
        global_acc, global_loss = test_model(test_features_flat, Y_val, global_model, comm_round)

comm_round: 0 | global_acc: 1.826% | global_loss: 3.850151300430298
comm_round: 1 | global_acc: 1.826% | global_loss: 3.8501527309417725
comm_round: 2 | global_acc: 1.826% | global_loss: 3.8501534461975098
comm_round: 3 | global_acc: 1.826% | global_loss: 3.850153684616089
comm_round: 4 | global_acc: 1.826% | global_loss: 3.850154399871826
comm_round: 5 | global_acc: 1.826% | global_loss: 3.850154399871826
comm_round: 6 | global_acc: 1.826% | global_loss: 3.850154399871826
comm_round: 7 | global_acc: 1.826% | global_loss: 3.8501551151275635
comm_round: 8 | global_acc: 1.826% | global_loss: 3.8501551151275635
comm_round: 9 | global_acc: 1.826% | global_loss: 3.8501551151275635
comm_round: 10 | global_acc: 1.826% | global_loss: 3.8501548767089844
comm_round: 11 | global_acc: 1.826% | global_loss: 3.8501548767089844
comm_round: 12 | global_acc: 1.826% | global_loss: 3.8501551151275635
comm_round: 13 | global_acc: 1.826% | global_loss: 3.8501548767089844
comm_round: 14 | global_acc: 1.826%